# Pricing of Maturity Monitored Barrier Security

This notebook prices the _"NBC Auto Callable Contingent Memory Income Note"_ described in this [link](https://nbcstructuredsolutions.ca/detailProduit.aspx?lequel=5317#documents).  This note is referred to as the _"security"_ for the remainder of this Notebook. 

This will be accomplished by first creating a function to generate the payout of the security if given the return of the underlying asset, which is the _Shares® Core S&P 500 Index ETF (CAD-Hedged)_.   This asset is referred to the _"ETF"_ for the remainder of the Notebook.

With this function defined we estimate the volatility of the ETF and then perform a Monte Carlo simulation of the ETF performance and the resulting security payout.  With the population of payout profiles we can estimate the fair price of the security.

## 1. Create the security payout function

#### Import some necessary modules.

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns

%matplotlib inline

#### Define some variables

In [3]:
barrier = -0.25      # Maturity-monitored barrier
coupon = 5.75 / 2    # Semi-annual coupon payment
call_threshold = 0.1 # Call threshold
face_value = 100     # Face value of the security
pf = 0.0             # Participation Factor
deferred_ppf = 0.05  # Deferred payment participation factor
current_ppf = 0.01   # Current payment participation factor
vrt = 0.0            # Variable return threshold

paydays = pd.to_datetime(['2020-08-18', '2021-02-18',
                          '2021-08-18', '2022-02-17',
                          '2022-08-18', '2023-02-17',
                          '2023-08-18', '2024-02-16',
                          '2024-08-19', '2025-02-18',
                          '2025-08-18', '2026-02-18',
                          '2026-08-18', '2027-02-18', '2027-02-25'], 
                           infer_datetime_format=True)

#### Calculate the discount factors for February 25 and March 3, 2020

These discount factors are for each of the 14 valuation dates plus the final maturity date relative to both February 25, 2020 and March 3, 2020.  The discount factors were calculated in the Excel file linked [here](fnen6800-pricing.xlsm).  These were based on Canadian bond price data from [MarketWatch](https://www.marketwatch.com/investing/bond) except for the 1-month T-Bill, whcih came from [YCharts](https://ycharts.com/indicators/canada_1_month_treasury_bill_yield).

In [7]:
z_feb = np.array([[0.992410841, 0.985339460, 0.979546758, 0.973836715, 0.968077416, 0.962380888, 0.956621589, 
                   0.951538723, 0.946359581, 0.941292119, 0.936256007, 0.930587457, 0.924659311, 0.918391061, 0.918150635]])

z_mar = np.array([[0.994351969, 0.988777333, 0.984887707, 0.981114970, 0.976878483, 0.972688169, 0.968451682, 
                   0.964645576, 0.960760469, 0.956959137, 0.952684688, 0.948130959, 0.942959266, 0.936977959, 0.936748540]])

#### Test the payout function with the examples given in this [link](https://nbcstructuredsolutions.ca/_afficheDocument.aspx?doc=05317_EN_CLIENT_SUMMARY.pdf&secur=0&rnd=0.01401764).

In [8]:
sample_returns = np.array([[ -9, -41, -12, -42,   -46, -48, -46.5, -50,   -47, -45.5, -47,   -43, -49,   -50],
                           [ -5,  -3,   6, -10,   -41, -46, -41,    -9,   -20, -49,   -45.5, -44, -41.5, -15],
                           [ -5, -10,  -7, -13.5, -46, -50, -47,   -46.5, -47, -42,   -44,   -14,  -4,    35],
                           [-10, -14,  -4, -13,   -11,  25,   1,    50,   -40, -60,    -5,     5,  95,     5]]).T/100.0

expected_payout = np.zeros((sample_returns.shape[0] + 1, sample_returns.shape[1]))
expected_payout[0, 0] = 2.875
expected_payout[2, 0] = 6.40
expected_payout[-1, 0] = 50

expected_payout[[0, 1, 3, 8], 1] = 2.875
expected_payout[2, 1] = 2.935
expected_payout[7, 1] = 12.3
expected_payout[-2, 1] = 14.875
expected_payout[-1, 1] = 100

expected_payout[[0, 1, 2, 3, -3], 2] = 2.875
expected_payout[-4, 2] = 23.55
expected_payout[-2, 2] = 3.225 + 100

expected_payout[0:5, 3] = 2.875
expected_payout[5, 3] = 3.125 + 100

print('Sample Returns:')
print(sample_returns)
print('\nExpected Payout:')
print(expected_payout)


Sample Returns:
[[-0.09  -0.05  -0.05  -0.1  ]
 [-0.41  -0.03  -0.1   -0.14 ]
 [-0.12   0.06  -0.07  -0.04 ]
 [-0.42  -0.1   -0.135 -0.13 ]
 [-0.46  -0.41  -0.46  -0.11 ]
 [-0.48  -0.46  -0.5    0.25 ]
 [-0.465 -0.41  -0.47   0.01 ]
 [-0.5   -0.09  -0.465  0.5  ]
 [-0.47  -0.2   -0.47  -0.4  ]
 [-0.455 -0.49  -0.42  -0.6  ]
 [-0.47  -0.455 -0.44  -0.05 ]
 [-0.43  -0.44  -0.14   0.05 ]
 [-0.49  -0.415 -0.04   0.95 ]
 [-0.5   -0.15   0.35   0.05 ]]

Expected Payout:
[[  2.875   2.875   2.875   2.875]
 [  0.      2.875   2.875   2.875]
 [  6.4     2.935   2.875   2.875]
 [  0.      2.875   2.875   2.875]
 [  0.      0.      0.      2.875]
 [  0.      0.      0.    103.125]
 [  0.      0.      0.      0.   ]
 [  0.     12.3     0.      0.   ]
 [  0.      2.875   0.      0.   ]
 [  0.      0.      0.      0.   ]
 [  0.      0.      0.      0.   ]
 [  0.      0.     23.55    0.   ]
 [  0.      0.      2.875   0.   ]
 [  0.     14.875 103.225   0.   ]
 [ 50.    100.      0.      0.   ]]


#### Define the payout function

In [9]:
def payout(returns, z):
    """
    Args:
        returns (np.array):  [n_dates, n_simulations] The ETF returns.
        z (np.array):  [1, n_dates + 1] Discount factors to be applied to the payout.
        
    Returns:
        np.array:  [n_dates + 1, n_simulations]  Payouts for valuation dates and maturity.
        np.array:  [n_simulations]  The present value of the payouts
    """
    n_dates = returns.shape[0]
    n_sims = returns.shape[1]

    # Determine the returns that are above the barrier (hits)
    hits = returns >= barrier
    miss = np.logical_not(hits)
    
    # Calculate the memory
    memory = np.zeros_like(returns)
    for i in range(1, n_dates):
        memory[i, :] = (memory[i - 1, :] + 1) * miss[i - 1, :]
    
    # Calculate deferred payment variable return if memory > 0
    deferred = (face_value * (returns - barrier) * deferred_ppf) * (memory > 0)
    
    # Calculate the Current Payment Variable Return
    variable = face_value * np.clip(returns - vrt, a_min=0.0, a_max=None) * current_ppf

    # Calculate the total coupon payment
    coupons = hits * (coupon * (memory + 1) + deferred) + variable
    del deferred, variable
    
    # Move all coupons upto call limit has been achieved to the payout array
    # Dummy created so argmax returns the size of the array if limit not volated
    payout = np.zeros((n_dates + 1, n_sims))
    dummy = np.vstack((returns, np.ones((1, n_sims)) * (call_threshold + 1)))
    hit_limit = np.argmax(dummy > call_threshold, axis=0) + 1
    del dummy
    for i in range(n_sims):
        h = min(hit_limit[i], n_dates)
        payout[:h, i] = coupons[:h, i]
    del coupons
    
    # Add the maturity Redemption Payment
    limit1 = min(call_threshold, 0.0)
    hit_limit -= 1
    for i in range(n_sims):
        if hit_limit[i] < n_dates:
            maturity_date_index = hit_limit[i]
            r = returns[hit_limit[i], i] 
        else:
            maturity_date_index = n_dates
            r = returns[-1, i] 

        if r > limit1:
            payout[maturity_date_index, i] = payout[maturity_date_index, i] + face_value * (1 + vrt)
        elif r >= barrier:
            payout[maturity_date_index, i] = payout[maturity_date_index, i] + face_value
        else:
            payout[maturity_date_index, i] = payout[maturity_date_index, i] + face_value * (1 + r)
    
    # Calculate present value of each simulation
    pv = z.dot(payout)
    
    return payout, pv[0, :]

# Run test
p, pv_feb = payout(sample_returns, z_feb)
if np.max(np.abs(p - expected_payout)) > 0:
    print("Coupon error detected in test set.")
    print(p - expected_payout)
else:
    print("Exact match with test set!!\n")

print("PV = [{}]".format(", ".join(["${:.2f}".format(v) for v in pv_feb])))


Exact match with test set!!

PV = [$55.03, $131.26, $130.68, $113.33]


## 2. Estimate ETF Volatility